# Image Captioning

This is the project for Deep Learning using Tensorflow: [Project Proposal](https://sharedcare.io/ImageCaptioning/proposal/)

In [1]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# PyDrive reference:
# https://googledrive.github.io/PyDrive/docs/build/html/index.html

# 2. Create & upload a file text file.
uploaded = drive.CreateFile({'title': 'Sample upload.txt'})
uploaded.SetContentString('Sample upload file content')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

# 3. Load a file by ID and print its contents.
downloaded = drive.CreateFile({'id': uploaded.get('id')})
print('Downloaded content "{}"'.format(downloaded.GetContentString()))

Uploaded file with ID 1izxKH2fQLVrHCRXWVWwbUjOhVuRes6VT
Downloaded content "Sample upload file content"


# Prepare the Training Data
##### Location to save the MSCOCO data.
MSCOCO_DIR="data/mscoco"

##### Run the preprocessing script.
sh ./download_and_preprocess_mscoco.sh \${MSCOCO_DIR}

##### Build the vocabulary.
DATA_DIR="mscoco/raw-data"

OUTPUT_DIR="mscoco/output"

python build_mscoco_data.py --train_image_dir=\${DATA_DIR}/train2014/ --val_image_dir=\${DATA_DIR}/val2014/ --train_captions_file=\${DATA_DIR}/annotations/captions_train2014.json --val_captions_file=\${DATA_DIR}/annotations/captions_val2014.json --output_dir=\${OUTPUT_DIR}/ --word_counts_output_file=\${OUTPUT_DIR}/word_counts.txt


# Download the Inception v3 Checkpoint
##### Location to save the Inception v3 checkpoint.
INCEPTION_DIR="data"

mkdir -p ${INCEPTION_DIR}

wget "http://download.tensorflow.org/models/inception_v3_2016_08_28.tar.gz"

tar -xvf "inception_v3_2016_08_28.tar.gz" -C ${INCEPTION_DIR}

rm "inception_v3_2016_08_28.tar.gz"

# Train a Model
##### Directory containing preprocessed MSCOCO data.
MSCOCO_DIR="data/data/mscoco"

##### Inception v3 checkpoint file.
INCEPTION_CHECKPOINT="data/inception_v3.ckpt"

##### Directory to save the model.
MODEL_DIR="model"

##### Train the model.
(tensorflow) device:im2txt user\$ python train.py --input_file_pattern="\${MSCOCO_DIR}/train-?????-of-00256" --inception_checkpoint_file="\${INCEPTION_CHECKPOINT}" --train_dir="\${MODEL_DIR}/train" --train_inception=false --number_of_steps=1000000